In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'

class AlexNetCustom(nn.Module):
    def __init__(self, num_classes=1000):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

class AlexNetCustom32(nn.Module):
    """Modified AlexNet for 32x32 input without upsampling"""
    def __init__(self, num_classes=100):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 3 * 3, 4096),  # Adjusted for smaller spatial size
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )
    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

print("="*70)
print("EXERCISE 1: AlexNet on CIFAR-100 - Upsampling vs Modified Architecture")
print("="*70)

transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

trainset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

model_upsampled = AlexNetCustom(num_classes=100).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_upsampled.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

print("\n[1] Training AlexNet with upsampling to 224x224...")
start_time = time.time()

for epoch in range(5):  # Reduced epochs for comparison
    model_upsampled.train()
    running_loss, correct, total = 0, 0, 0
    for inputs, labels in tqdm(trainloader, desc=f'Epoch {epoch+1}/5'):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_upsampled(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    scheduler.step()
    print(f'Epoch {epoch+1} | Loss: {running_loss/len(trainloader):.3f} | Acc: {100*correct/total:.2f}%')

time_upsampled = time.time() - start_time
print(f"\nTime for upsampled version: {time_upsampled:.2f} seconds")

# Approach 2: Modified architecture for 32x32
transform_train_32 = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

trainset_32 = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train_32)
trainloader_32 = DataLoader(trainset_32, batch_size=128, shuffle=True, num_workers=2)  # Larger batch possible

model_modified = AlexNetCustom32(num_classes=100).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_modified.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

print("\n[2] Training modified AlexNet for 32x32 input...")
start_time = time.time()

for epoch in range(5):
    model_modified.train()
    running_loss, correct, total = 0, 0, 0
    for inputs, labels in tqdm(trainloader_32, desc=f'Epoch {epoch+1}/5'):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_modified(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    scheduler.step()
    print(f'Epoch {epoch+1} | Loss: {running_loss/len(trainloader_32):.3f} | Acc: {100*correct/total:.2f}%')

time_modified = time.time() - start_time
print(f"\nTime for modified version: {time_modified:.2f} seconds")
print(f"\nSpeedup: {time_upsampled/time_modified:.2f}x")

EXERCISE 1: AlexNet on CIFAR-100 - Upsampling vs Modified Architecture

[1] Training AlexNet with upsampling to 224x224...


Epoch 1/5:   1%|▏         | 11/782 [01:14<1:26:30,  6.73s/it]

In [ ]:
print("\n" + "="*70)
print("EXERCISE 2: Replace ReLU with GELU")
print("="*70)

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,32,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*8*8, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

class SimpleCNN_GELU(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3,32,3,padding=1),
            nn.GELU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32,64,3,padding=1),
            nn.GELU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*8*8, 256),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914,0.4822,0.4465),(0.2023,0.1994,0.2010))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

print("\n[1] Training SimpleCNN with ReLU...")
model_relu = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_relu.parameters(), lr=1e-3)

for epoch in range(10):
    model_relu.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_relu(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, loss {running_loss/len(trainloader):.4f}')

model_relu.eval()
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_relu(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
acc_relu = 100 * correct / total
print(f'ReLU Test Accuracy: {acc_relu:.2f}%')

print("\n[2] Training SimpleCNN with GELU...")
model_gelu = SimpleCNN_GELU().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_gelu.parameters(), lr=1e-3)

for epoch in range(10):
    model_gelu.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_gelu(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, loss {running_loss/len(trainloader):.4f}')

model_gelu.eval()
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_gelu(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
acc_gelu = 100 * correct / total
print(f'GELU Test Accuracy: {acc_gelu:.2f}%')

print(f"\nComparison: GELU vs ReLU = {acc_gelu - acc_relu:+.2f}% difference")

In [ ]:
print("\n" + "="*70)
print("EXERCISE 3: Implement residual blocks and create tiny-ResNet")
print("="*70)


class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = torch.relu(out)
        return out


class TinyResNet(nn.Module):
    """Based on SimpleCNN structure but with residual blocks"""
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)


        self.layer1 = ResidualBlock(32, 32, stride=1)
        self.pool1 = nn.MaxPool2d(2)

        self.layer2 = ResidualBlock(32, 64, stride=1)
        self.pool2 = nn.MaxPool2d(2)


        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*8*8, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = torch.relu(self.bn1(self.conv1(x)))
        x = self.pool1(self.layer1(x))
        x = self.pool2(self.layer2(x))
        x = self.classifier(x)
        return x


model_resnet = TinyResNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_resnet.parameters(), lr=1e-3)

print("\nTraining Tiny-ResNet...")
for epoch in range(10):
    model_resnet.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model_resnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, loss {running_loss/len(trainloader):.4f}')


model_resnet.eval()
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_resnet(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
print(f'Tiny-ResNet Test Accuracy: {100*correct/total:.2f}%')


In [ ]:

print("\n" + "="*70)
print("EXERCISE 4: Channel Pruning")
print("="*70)

print("\nOriginal model accuracy (ReLU):", f'{acc_relu:.2f}%')

def prune_model(model, prune_ratio=0.3):
    """Prune channels with lowest L1 norm from convolutional layers"""
    print(f"\nPruning {prune_ratio*100:.0f}% of channels...")

    conv1 = model.features[0]
    weights = conv1.weight.data

    l1_norm = torch.sum(torch.abs(weights), dim=(1,2,3))

    num_channels = l1_norm.shape[0]
    num_keep = int(num_channels * (1 - prune_ratio))

    _, indices = torch.sort(l1_norm, descending=True)
    keep_indices = indices[:num_keep].sort()[0]

    print(f"Conv layer: {num_channels} channels -> {num_keep} channels")

    pruned_weights = weights.clone()
    prune_indices = indices[num_keep:]
    pruned_weights[prune_indices] = 0

    conv1.weight.data = pruned_weights

    return model

model_pruned = prune_model(model_relu, prune_ratio=0.3)

model_pruned.eval()
correct, total = 0, 0
with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_pruned(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
acc_pruned = 100 * correct / total

print(f'\nPruned model accuracy: {acc_pruned:.2f}%')
print(f'Accuracy drop: {acc_relu - acc_pruned:.2f}%')
print(f'Note: For actual parameter reduction, rebuild model with fewer channels')

print("\n" + "="*70)
print("All exercises completed!")
print("="*70)